In [ ]:
from qiskit import QuantumCircuit, qasm3
from quantum_chip import QuantumChip
from hardware import HardwareParams
from qrmove_compiler import QRMoveCompiler
from utils import get_quantum_circuit
import math
import signal

In [ ]:
# Default Settings
# quantum_alg: bv
# MRP_CP_ratio: 10
# extra_qubit_ratio: 15%
# qubit_num: 30


class TimeoutError(Exception):
    pass


def timeout_handler(signum, frame):
    raise TimeoutError("Function call timed out")


def experiment(quantum_alg, qubit_num, extra_qubit_num, MRP_CP_ratio):
    """
    quantum_alg: 量子算法
    qubit_num: 量子比特数量
    extra_qubit_ratio: 额外比特比例
    MRP_CP_ratio: MRP/CP 比例
    """
    # 设置信号处理器和超时时间（5分钟）
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(5 * 60)

    try:
        quantum_circuit: QuantumCircuit = get_quantum_circuit(quantum_alg, qubit_num)
        quantum_chip = QuantumChip("square", 50)
        hardware_param = HardwareParams(
            time_1q=300.0,  # 单比特门时间
            time_2q=300.0,  # 双比特门时间
            time_meas=300 * MRP_CP_ratio,  # 测量时间
            time_reset=0,  # 重置时间
        )
        qmc = QRMoveCompiler(
            quantum_circuit,
            quantum_chip,
            hardware_param,
            math.ceil(extra_qubit_num),
        )
        (
            start_depth,
            start_width,
            qr_map_depth,
            qr_map_width,
            qr_move_depth,
            qr_move_width,
        ) = qmc.compile_program()

        # 取消警报
        signal.alarm(0)
        return (
            start_depth,
            start_width,
            qr_map_depth,
            qr_map_width,
            qr_move_depth,
            qr_move_width,
        )

    except (TimeoutError, Exception) as e:
        # 取消警报以防万一
        signal.alarm(0)
        print(f"Experiment failed with error: {e}")
        return 0, 0, 0

In [ ]:
def experiment_var_extra_ratio_alg():
    for alg in ["bv", "vqa", "qaoa", "qft", "mul", "rd", "xor"]:
        for extra_ratio in [0.1, 0.2, 0.3]:
            qubit_num = 30
            print(
                f"-- 参数: {alg}算法, 额外比特比例 {extra_ratio}, 量子比特数量 {qubit_num} --"
            )
            print(alg, extra_ratio, qubit_num)
            for _ in range(50):
                (
                    start_depth,
                    start_width,
                    qr_map_depth,
                    qr_map_width,
                    qr_move_depth,
                    qr_move_width,
                ) = experiment(alg, qubit_num, extra_ratio * qubit_num, 10)
                print(start_depth, qr_map_depth, qr_move_depth)
                print(qubit_num, qr_map_depth, qr_move_depth)


experiment_var_extra_ratio_alg()

In [ ]:
def experiment_var_qubit_num():
    for qubit_num in range([10, 20, 30, 40, 50]):
        for alg in ["bv", "vqa", "qaoa", "qft", "mul", "rd", "xor"]:
            MRP_CP_ratio = 10
            extra_qubit_ratio = 0.15
            (
                start_depth,
                start_width,
                qr_map_depth,
                qr_map_width,
                qr_move_depth,
                qr_move_width,
            ) = experiment(alg, qubit_num, 0.15 * qubit_num, MRP_CP_ratio)
            print(alg, extra_ratio, qubit_num)
            print(start_depth, qr_map_depth, qr_move_depth)


experiment_var_qubit_num()

In [ ]:
def experiment_var_mrp_ratio():
    for i in range([5, 10, 15, 20, 25]):
        MRP_CP_ratio = i
        for _ in range(50):
            (
                start_depth,
                start_width,
                qr_map_depth,
                qr_map_width,
                qr_move_depth,
                qr_move_width,
            ) = experiment("bv", 50, 0.15 * 50, MRP_CP_ratio)
            print(alg, extra_ratio, qubit_num)
            print(start_depth, qr_map_depth, qr_move_depth)


experiment_var_mrp_ratio()

In [ ]:
def experiment_var_width_depth():
    qubit_num = 30
    MRP_CP_ratio = 10
    alg = "bv"
    for extra_qubit in range(30):
        (
            start_depth,
            start_width,
            qr_map_depth,
            qr_map_width,
            qr_move_depth,
            qr_move_width,
        ) = experiment(alg, qubit_num, 0.15 * qubit_num, MRP_CP_ratio)
        print(alg, extra_qubit, qubit_num)
        print(start_depth, qr_map_depth, qr_move_depth)


experiment_var_width_depth()